# Accuracy baselines

Computation of the insertion and deletion scores on a testbench of explanations for 100 images of the ImageNet training dataset.

In [1]:
!pip install xplique
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2
import pandas as pd
import os
from google.colab import drive
import json
import itertools

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
from xplique.attributions import SobolAttributionMethod, Rise # black box methods
from xplique.attributions import Saliency, GradientInput, IntegratedGradients, GradCAMPP, VarGrad# white box methods
from xplique.metrics import Deletion, Insertion, MuFidelity # evaluation methods
from spectral_sobol.tf_explainer import WaveletSobol
import keras.applications as app # models

In [3]:
# load the labels and the images
source_dir = '/content/gdrive/MyDrive/sam-benchmark/imagenet'
target_dir = '/content/gdrive/MyDrive/sam-benchmark'

# number of iterations to compute the scores
reps = 1
nb_design = 4
grid_size = 28
batch_size = 128

# labels
labels = pd.read_csv(os.path.join(source_dir, 'labels.csv'))
y_wavelet = labels['label'].values
y = np.array([tf.keras.utils.to_categorical(label, 1000) for label in y_wavelet])

# images
images = [cv2.imread(os.path.join(source_dir, img_name))[..., ::-1] for img_name in labels['name'].values]


# helper that will update the file if it exists already
def update(target_dir, filename, data):
  """
  updates the file if it exists, creates and save otherwise
  """

  exists = os.path.exists(os.path.join(target_dir, filename))

  if exists:
    old = json.load(open(os.path.join(target_dir, filename)))
    
    # add the items
    for item in data.keys():
      old[item] = data[item]

    # save the file
    with open(os.path.join(target_dir, filename), 'w') as f:
      json.dump(old, f)

  else:
    with open(os.path.join(target_dir, filename), 'w') as f:
      json.dump(data, f)

# model set up
# each value contains the model and its preprocessing module
models = {
    'VGG'          : (app.VGG16(classifier_activation = 'softmax'), app.vgg16.preprocess_input),
    'ResNet'       : (app.ResNet50(classifier_activation = 'softmax'), app.resnet.preprocess_input),
    'MobileNet'    : (app.MobileNet(classifier_activation = 'softmax'), app.mobilenet.preprocess_input),
    'EfficientNet' : (app.EfficientNetV2B0(classifier_activation = 'softmax'), app.efficientnet.preprocess_input)
}

29403144/29403144 [==============================] - 1s 0us/step


In [7]:
# computation

scores = {}

for name in models.keys():

  print('Starting model {}'.format(name))

  model, preprocessing = models[name]
  model.layers[-1].activation = tf.keras.activations.softmax # 

  # preprocess the data
  x = preprocessing(np.array(images, copy=True))

  # initialize the metrics
  insertion = Insertion(model, x, y)
  deletion = Deletion(model, x, y)

  # set up the explanations
  explainers = {
      'saliency'   : Saliency(model),
      'grad-inp'   : GradientInput(model),
      'grad-integ' : IntegratedGradients(model, steps=80),
      'GradCAMPP'  : GradCAMPP(model),
      'VarGrad'    : VarGrad(model, nb_samples=80),
      'rise'  : Rise(model, nb_samples=4000),
      'sobol' : SobolAttributionMethod(model)
  }

  print('Explanations set up complete.')

  scores[name] = {}

  # compute the different explanations for this backbone
  for explanation in explainers.keys():

    explainer = explainers[explanation]
    out = explainer(x,y)
    
    scores[name][explanation] = {
        'insertion' : insertion(out),
        'deletion'  : deletion(out)
    }

    print(scores)

    update(target_dir, 'benchmark.json', scores)
    print('Computation and evaluation of the explanation {} complete.'.format(explanation))


  print('Model {} complete'.format(name))



Starting model VGG
Explanations set up complete.
{'VGG': {'saliency': {'insertion': 0.21933262050151825, 'deletion': 0.10000298172235489}}}
Computation and evaluation of the explanation saliency complete.
{'VGG': {'saliency': {'insertion': 0.21933262050151825, 'deletion': 0.10000298172235489}, 'grad-inp': {'insertion': 0.14044031500816345, 'deletion': 0.049855802208185196}}}
Computation and evaluation of the explanation grad-inp complete.
{'VGG': {'saliency': {'insertion': 0.21933262050151825, 'deletion': 0.10000298172235489}, 'grad-inp': {'insertion': 0.14044031500816345, 'deletion': 0.049855802208185196}, 'grad-integ': {'insertion': 0.1712673008441925, 'deletion': 0.0414176844060421}}}
Computation and evaluation of the explanation grad-integ complete.
{'VGG': {'saliency': {'insertion': 0.21933262050151825, 'deletion': 0.10000298172235489}, 'grad-inp': {'insertion': 0.14044031500816345, 'deletion': 0.049855802208185196}, 'grad-integ': {'insertion': 0.1712673008441925, 'deletion': 0.04

{'VGG': {'saliency': {'insertion': 0.21933262050151825, 'deletion': 0.10000298172235489}, 'grad-inp': {'insertion': 0.14044031500816345, 'deletion': 0.049855802208185196}, 'grad-integ': {'insertion': 0.1712673008441925, 'deletion': 0.0414176844060421}, 'GradCAMPP': {'insertion': 0.3988441526889801, 'deletion': 0.11016499996185303}, 'VarGrad': {'insertion': 0.2226378172636032, 'deletion': 0.14782145619392395}, 'rise': {'insertion': 0.4601594805717468, 'deletion': 0.1050887331366539}, 'sobol': {'insertion': 0.37721818685531616, 'deletion': 0.10956649482250214}}, 'ResNet': {'saliency': {'insertion': 0.2321566343307495, 'deletion': 0.12438156455755234}, 'grad-inp': {'insertion': 0.13432364165782928, 'deletion': 0.08310393989086151}, 'grad-integ': {'insertion': 0.17016859352588654, 'deletion': 0.07145336270332336}}}
Computation and evaluation of the explanation grad-integ complete.
{'VGG': {'saliency': {'insertion': 0.21933262050151825, 'deletion': 0.10000298172235489}, 'grad-inp': {'insert

{'VGG': {'saliency': {'insertion': 0.21933262050151825, 'deletion': 0.10000298172235489}, 'grad-inp': {'insertion': 0.14044031500816345, 'deletion': 0.049855802208185196}, 'grad-integ': {'insertion': 0.1712673008441925, 'deletion': 0.0414176844060421}, 'GradCAMPP': {'insertion': 0.3988441526889801, 'deletion': 0.11016499996185303}, 'VarGrad': {'insertion': 0.2226378172636032, 'deletion': 0.14782145619392395}, 'rise': {'insertion': 0.4601594805717468, 'deletion': 0.1050887331366539}, 'sobol': {'insertion': 0.37721818685531616, 'deletion': 0.10956649482250214}}, 'ResNet': {'saliency': {'insertion': 0.2321566343307495, 'deletion': 0.12438156455755234}, 'grad-inp': {'insertion': 0.13432364165782928, 'deletion': 0.08310393989086151}, 'grad-integ': {'insertion': 0.17016859352588654, 'deletion': 0.07145336270332336}, 'GradCAMPP': {'insertion': 0.4480069577693939, 'deletion': 0.1831328570842743}, 'VarGrad': {'insertion': 0.2571285665035248, 'deletion': 0.17595311999320984}, 'rise': {'insertion

{'VGG': {'saliency': {'insertion': 0.21933262050151825, 'deletion': 0.10000298172235489}, 'grad-inp': {'insertion': 0.14044031500816345, 'deletion': 0.049855802208185196}, 'grad-integ': {'insertion': 0.1712673008441925, 'deletion': 0.0414176844060421}, 'GradCAMPP': {'insertion': 0.3988441526889801, 'deletion': 0.11016499996185303}, 'VarGrad': {'insertion': 0.2226378172636032, 'deletion': 0.14782145619392395}, 'rise': {'insertion': 0.4601594805717468, 'deletion': 0.1050887331366539}, 'sobol': {'insertion': 0.37721818685531616, 'deletion': 0.10956649482250214}}, 'ResNet': {'saliency': {'insertion': 0.2321566343307495, 'deletion': 0.12438156455755234}, 'grad-inp': {'insertion': 0.13432364165782928, 'deletion': 0.08310393989086151}, 'grad-integ': {'insertion': 0.17016859352588654, 'deletion': 0.07145336270332336}, 'GradCAMPP': {'insertion': 0.4480069577693939, 'deletion': 0.1831328570842743}, 'VarGrad': {'insertion': 0.2571285665035248, 'deletion': 0.17595311999320984}, 'rise': {'insertion